In [ ]:
import keyring        # for loading api token
import urllib.request # for encoding URL parameters
import pandas as pd   # for handling data frames
import json           # for handling json
import os             # for outputting the absolute path of the file containing the data

# Authentication

Please read the Authentication section of the [README](README.ipynb) to set up your acess token for the following code.  <span style="color:red"> WARNING: the below code will load the access token you saved from the README.ipynb</span>

In [ ]:
token = keyring.get_password("system", "canvas_token");
print("Loaded token with %d characters." % (len(token)))

# Configuration

The following configuration does not frequently change. Input your course ID found in the url of your canvas course.

In [ ]:
course_id = int(input("Enter your course id here: "))

This next one can change based on your rubric.  This can be found from the data returned from the assignments notebook or by looking at the outcomes section of canvas.

In [ ]:
rubric_id = int(input("Enter your rubric id here: "))

In [ ]:
config = {
        'base_url': 'https://usfca.instructure.com',
        'course_id': course_id,
        'rubric': rubric_id
}

# Request Setup

The following sets up the REST API request to get a rubric:

<https://canvas.instructure.com/doc/api/rubrics.html#method.rubrics_api.show>

In [ ]:
api_format = '{base_url}/api/v1/courses/{course_id}/rubrics/{rubric}'
api_url = api_format.format(**config)
print("URL:", api_url)

In [ ]:
params = {
    'per_page': 200,
    'include[]': 'assessments',
    'style':'full'
}

encoded = urllib.parse.urlencode(params)
print("Params:", encoded) # do not output api key 

In [ ]:
rest_call = '{}?access_token={}&{}'.format(api_url, token, encoded)
print("REST call is %d characters." % (len(rest_call)))

# Fetch Data

Fetch the JSON data from the API.

In [ ]:
# due to the format of this json object, some preprocessing using the json library needs to be done then converted to a dataframe
with urllib.request.urlopen(rest_call) as url:
    data = json.loads(url.read().decode())
data = pd.DataFrame.from_dict(data, orient='index')
print('Loaded {} rows and {} columns.'.format(*data.shape))
# The column should be the rubric 
# the rows are the information about the rubric

In [ ]:
# output columns (should be one due to requesting one rubric)
print('Columns:', list(data.columns.values))

In [ ]:
data.head() # look at the results

# Obtain Questions

Attempting to obtain question information if there is one in the assignment.  These questions correspond to the questions on a rubric.

In [ ]:
try:
    questions = data.loc['criteria']
except KeyError:
    print("There is no question information here please input a new assignment or check that your access token is accurate")

In [ ]:
converted = pd.DataFrame.from_dict(questions) # we only need the criteria from the data gathered from the api

# Create Criteria Dataframe

Creating dataframe out of the criteria 

In [ ]:
expanded = converted.explode("criteria") # converted is a single row containing a list of all questions of a rubric
expanded.reset_index(inplace=True, drop=True)

In [ ]:
expanded.head() # look at the results

In [ ]:
expanded.loc[0, "criteria"] # look at a single question of a rubric

# Wrangling

Some of the columns could use some wrangling.

In [ ]:
wrangled = expanded.copy()

In [ ]:
# convert the dictionary that corresponds to the question into a row of a dataframe for all rows
criteria = wrangled.criteria.apply(pd.Series)

In [ ]:
criteria.head()

In [ ]:
# there are many similar names with each endpoint of the data these two columns in particular could be more clear
criteria.rename(columns={"id":"criterion_id", "points":"points_possible"}, inplace=True) 

In [ ]:
ratings = criteria.explode("ratings") # gets the ratings (this is the information for ratings within the question)

In [ ]:
ratings = ratings.ratings.apply(pd.Series)

In [ ]:
ratings.columns = ["ratings_" + item for item in ratings.columns.tolist()] # some columns may have the same name so change them

In [ ]:
final = pd.concat([criteria, ratings], axis=1) # combine both datasets

In [ ]:
final.head() # note that here you may have duplicate rows due to the fact that each question has a choice of multiple ratings

In [ ]:
final.reset_index(inplace=True)

In [ ]:
final.drop(columns=["ratings"], inplace=True) # since we combined the rating information, we can safely drop the column without losing information

# Optional

Do this if you already saved a set of questions to a file and want to add another set of questions to it from another assignment

In [ ]:
# questions = pd.read_csv('questions.csv')
# final = pd.concat([final, questions], axis=0, sort=False)

# Output Results

Output the results to CSV.

In [ ]:
path = 'questions.csv'

In [ ]:
final.to_csv(path, header = True, index = False)

In [ ]:
os.path.abspath('questions.csv') # the absolute path of the result